In [1]:
import pandas as pd
import numpy as np

C:\Users\vunv\AppData\Local\Temp\ipykernel_16152\2162656668.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [31]:
DATA = []

In [32]:
import json


def get_data_from_file(filename: str):
    f = open(filename, "r")
    while True:
        line = f.readline()
        if not line:
            break
        # Standardlize json to dict
        row = json.loads(line)
        # Append to Array
        DATA.append(row)

In [33]:
from itertools import count


get_data_from_file("../statics/request_data_with_duration_time.json")
print(len(DATA))

14798


In [199]:
# Create dataframe
root_df = pd.DataFrame(DATA)

In [200]:
root_df

,timestamp,path,method,service_name,controller_pod,duration_time
0,"23/01/2024, 10:44:34",/users,GET,process_log,pod-1,0.176
1,"23/01/2024, 10:44:34",/home,GET,process_log,pod-2,0.654
2,"23/01/2024, 10:44:34",/users,GET,process_log,pod-1,0.562
3,"23/01/2024, 10:44:34",/users,GET,process_log,pod-1,0.690
4,"23/01/2024, 10:44:35",/users,GET,process_log,pod-1,0.333
...,...,...,...,...,...,...
14793,"23/01/2024, 11:34:26",/home,GET,process_log,pod-1,0.148
14794,"23/01/2024, 11:34:26",/users,GET,process_log,pod-1,0.145
14795,"23/01/2024, 11:34:26",/users,GET,process_log,pod-2,0.298
14796,"23/01/2024, 11:34:26",/users,GET,process_log,pod-1,0.243


In [201]:
# Create dataset bucket
# Bucket by le
# value[le="0.005"]
# value[le="0.01"]
# value[le="0.025"]
# value[le="0.05"]
# value[le="0.1"]
# value[le="0.25"]
# value[le="0.5"]
# value[le="+Inf"]

In [202]:
buckets = [0.005, 0.01, 0.025, 0.05, 0.1, 0.25, 0.5]

def bucketing(duration_time):
    res = []
    for bucket in buckets:
        if duration_time <= bucket:
            res.append(bucket)
    res.append("+Inf")
    return res

In [203]:
root_df["le"] = root_df["duration_time"].apply(lambda x: bucketing(x))

In [204]:
# Re format timestamp
from datetime import datetime
root_df['timestamp'] = root_df['timestamp'].apply(lambda x: datetime.strptime(x, "%d/%m/%Y, %H:%M:%S"))

In [237]:
from typing import Dict


def bins_value(row, bins_dict: Dict):
    # start to bin data in each group
    list_le = row['le']
    for le in list_le:
        if le in bins_dict:
            bins_dict[le] += 1
    bins_series = pd.Series(
        data=bins_dict, index=bins_dict.keys()  # type: ignore
    )
    # Join two series
    new_row = pd.concat([row, bins_series])
    return new_row


# root_df = root_df.apply(lambda x: bins_value(x), axis=1)  # type: ignore

In [238]:
# get distinct group
group_dataset = root_df.groupby(['path', 'method', 'service_name', 'controller_pod']).count().reset_index()[['path', 'method', 'service_name', 'controller_pod']]

In [239]:
group_dataset

,path,method,service_name,controller_pod
0,/home,GET,process_log,pod-1
1,/home,GET,process_log,pod-2
2,/users,GET,process_log,pod-1
3,/users,GET,process_log,pod-2


In [ ]:
df_bin_by_group = pd.DataFrame()
row_data_by_group = {}
for idx, row in group_dataset.iterrows():
    print("Path: ", row['path'])
    bins_dict = {k: 0 for k in buckets}
    bins_dict['+Inf'] = 0  # type: ignore
    row_data_by_group[idx] = root_df.query(
        "path == @row['path'] and method == @row['method'] and service_name == @row['service_name'] and controller_pod == @row['controller_pod']").apply(lambda x: bins_value(x, bins_dict), axis=1)

In [250]:
# Concat group data to dataframe
df_bin_by_group = pd.concat(row_data_by_group.values())
print(len(row_data_by_group))

4


In [252]:
df_bin_by_group[100:130]

,timestamp,path,method,service_name,controller_pod,duration_time,le,0.005,0.01,0.025,0.05,0.1,0.25,0.5,+Inf
454,2024-01-23 10:46:05,/home,GET,process_log,pod-1,0.658,[+Inf],0,0,0,2,9,32,62,101
455,2024-01-23 10:46:06,/home,GET,process_log,pod-1,0.765,[+Inf],0,0,0,2,9,32,62,102
465,2024-01-23 10:46:08,/home,GET,process_log,pod-1,0.683,[+Inf],0,0,0,2,9,32,62,103
472,2024-01-23 10:46:09,/home,GET,process_log,pod-1,0.423,"[0.5, +Inf]",0,0,0,2,9,32,63,104
474,2024-01-23 10:46:10,/home,GET,process_log,pod-1,0.377,"[0.5, +Inf]",0,0,0,2,9,32,64,105
478,2024-01-23 10:46:10,/home,GET,process_log,pod-1,0.460,"[0.5, +Inf]",0,0,0,2,9,32,65,106
480,2024-01-23 10:46:11,/home,GET,process_log,pod-1,0.052,"[0.1, 0.25, 0.5, +Inf]",0,0,0,2,10,33,66,107
484,2024-01-23 10:46:12,/home,GET,process_log,pod-1,0.691,[+Inf],0,0,0,2,10,33,66,108
486,2024-01-23 10:46:12,/home,GET,process_log,pod-1,0.756,[+Inf],0,0,0,2,10,33,66,109
488,2024-01-23 10:46:12,/home,GET,process_log,pod-1,0.243,"[0.25, 0.5, +Inf]",0,0,0,2,10,34,67,110
